In [1]:

from bs4 import BeautifulSoup
import requests
from googlesearch import search
query = "esg sector :pdf"
import requests
from bs4 import BeautifulSoup
from boilerpipe.extract import Extractor
import time
from selenium import webdriver
import csv

# URL='https://www.beautypackaging.com/contents/view_breaking-news/2021-12-13/geka-earns-b-from-cdp-for-reducing-its-environmental-impact/'

# extractor = Extractor(extractor='ArticleExtractor', url=URL)
# print(extractor.getText())
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"

# Beautiful Soup implementation
def get_soup_from_BeautifulSoup(params, *headers):
    response = requests.get("https://www.google.com/search", params=params, headers=headers )
    
    return BeautifulSoup(response.text, 'lxml')

# Selenium Implementation
def get_html_source(url, sleep_time=10):
    browser = webdriver.Chrome()
    # get web page
    browser.get(url)
    # execute script to scroll down the page
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    # sleep for time
    time.sleep(sleep_time)
    return browser.page_source

def get_google_url(params):
    params_key, params_value = list(params.keys()),list(params.values())
    url = f"https://www.google.com/search?"
    for i in range(len(params)):
        url = url + params_key[i] + "=" + str(params_value[i]).replace(" ", "+")+ "&"
    return url

def get_soup_from_Selenium(params, *sleep_time):
    if not sleep_time:
        sleep_time = (10,)
    params_key, params_value = list(params.keys()),list(params.values())
    url = f"https://www.google.com/search?"
    for i in range(len(params)):
        url = url + params_key[i] + "=" + str(params_value[i]).replace(" ", "+")+ "&"
    return BeautifulSoup(get_html_source(url, sleep_time[0]), 'html.parser')

# --------------------------------------------------

Change Items in This Cell

In [2]:
header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
    'referer':'https://www.google.com/'
}
# Google Search Parameters LIST https://stenevang.wordpress.com/2013/02/22/google-advanced-power-search-url-request-parameters/
params = {
    "q": 'new climate protocols "$"',
    "hl": "en",
    "tbm": "nws",
    "num": 100, #no. of results per page
    #  "tbs": "sbd:1", #Sort by Date
    "start": None,
}
page_number = 1 #change the page number from here
params["start"] = params["num"]*(page_number-1)
time.sleep(1.2)

soup = get_soup_from_Selenium(params, 5)
# soup = get_soup_from_BeautifulSoup(params)

with open("test.html", mode="w+") as f:
    f.write(soup.prettify())
    f.close()

In [30]:
# Using Chrome Web Driver

def get_data_list_using_chrome_driver(soup):
    
    g_cards = soup.find_all("g-card", id="")
    all_info = []
    # iteration = 0
    for a_card in g_cards:
        link_post = a_card.find("a")["href"]
        data_list = list(a_card.strings)
        title = data_list[1]
        title = title.replace("...", ".")
        title = title.replace("\n", "")
        description = data_list[2]
        description = description.replace("\n", "")
        description = description.replace("...", ".")
        publish_date = data_list[4]
        _info = {
            "link_post":link_post,
            "title":title,
            "description":description,
            "publish_date":publish_date,
            "article": ""
            }
        try:
            response = requests.get(_info["link_post"], timeout=30)
            _info["article"] = Extractor(extractor='ArticleExtractor', html=response.text).getText()
            
            
        except Exception as e:
            print("fail for", _info["link_post"])
            _info["article"] = "Error"
            print(e)
        all_info.append(_info)
        # iteration +=1
        # print(iteration)
    return all_info
the_data = get_data_list_using_chrome_driver(soup)
the_data

def csv_write_data(list_of_data, file_name, write_mode):
    with open(file_name, write_mode, encoding="utf-8") as file:
        csv_writer = csv.DictWriter(file, fieldnames=list_of_data[0].keys())
        csv_writer.writeheader()
        for line in list_of_data:
            try:
                csv_writer.writerow(line)
            except Exception as error:
                print("Error in writing to CSV", error)
        file.close()
    return

csv_write_data(the_data,"mydata.csv","w")

file =  open("mydata.csv", "r",encoding ="utf-8")
csv_reader = list(csv.DictReader(file))
file.close()
# list(csv_reader)

fail for https://www.seattletimes.com/sports/kraken/kraken-blue-jackets-notes-from-geoff-baker-for-sunday-print/
HTTPSConnectionPool(host='www.seattletimes.com', port=443): Read timed out. (read timeout=30)


Retailer
ACE’s Low Carbon Strategy Pays Dividends
A new USDA program could soon establish the quantification and verification protocols necessary for long-awaited climate-smart farming incentives, which could begin to scale ethanol’s role in an increasingly  decarbonized transportation sector.
By Brian Jennings       | December 20, 2021
ADVERTISEMENT
Given persistent challenges regarding the Environmental Protection Agency’s mismanagement of the RFS, ACE has been taking direction from our Board of Directors to be proactive about opportunities to expand ethanol demand through new clean fuel or low carbon policies and markets.
According to the United States Department of Agriculture, U.S. farmers currently store over 20 million metric tons of carbon per year and they can store an additional 180 million metric tons per year representing 12-14 percent of U.S. carbon emissions annually. The Department of Energy’s Argonne National Laboratory has found that deploying specific crop rotation sy

In [ ]:
# using Beautiful Soup
# The main content of google search
def get_data_list_using_BeautifulSoup(soup):
    main_div = soup.find("div", id="main")
    # making list of all div tags
    one_before_start_div = main_div.findChild("div")
    one_before_start_div = one_before_start_div.find_next_siblings("div")
    one_before_start_div = one_before_start_div[1]

    list_of_divs = one_before_start_div.find_next_siblings("div")

    # Clearing the memory
    # del one_before_start_div, main_div
    # Making list of all the data on the google search page
    all_info = []
    for the_div in list_of_divs:
        link_to_post = the_div.find_next("a").attrs["href"]
        if link_to_post[0] == "/":
            link_to_post = "https://www.google.com" + link_to_post

        info_list = the_div.find_next("div").find_next("div").find_next_sibling().find_next_sibling()
        info_list = info_list.find_next("span").parent.find_all(string=True)
        
        title = the_div.find_next("h3").text
        # Removes an extra dot in list
        info_list.pop(1)
        info_list.insert(0, title)
        # words_date = info_list[0].split(" ")
        # if int(words_date[0]) > 4 and words_date[1] == "months": #If the news is from before 4 months it wont be saved into list
        #     continue
        # datetime_object = datetime.strptime(info_list[0], '%b %d %Y %I:%M%p')
        # print(datetime_object)
        # print(link_to_post)
        # print(info_list)
        _info = [link_to_post]
        # OPTIONAL updating the redirected URL to full URL, this will impact performance!!
        # _info[0] = requests.get(_info[0],headers=headers).url
        _info.extend(info_list)
        dict_info = {
            "link_post":_info[0],
            "title":_info[1],
            "description":_info[2],
            "publish_date":_info[3]
        }
        # appending the info into the list
        all_info.append(dict_info)
        # order in final list is url, published at, description
    return all_info
# all_info = get_data_list_using_BeautifulSoup(soup)
# all_info
# print(Extractor(extractor='ArticleExtractor', url=myurl.url).getText())
